In [19]:
import numpy as np

In [20]:
def readArchive(fileName):
    with open(fileName, 'r') as file:
        lines = file.readlines()

    # Elimina los caracteres de salto de línea y divide los valores por comas
    data = [line.strip().split(',') for line in lines]

    # Convierte la lista en una matriz de numpy
    return np.array(data, dtype=int)

dataTrain = readArchive('optdigits.tra')

X_train = dataTrain[:, :64]
Y_train = dataTrain[:, 64]

dataTest = readArchive('optdigits.tes')
X_test = dataTest[:, :64]
Y_test = dataTest[: , 64]

In [21]:
def calculate_confusion_matrix(true_labels, predicted_labels, num_classes):
    confusion_matrix = np.zeros((num_classes, num_classes))
    for true_label, predicted_label in zip(true_labels, predicted_labels):
        confusion_matrix[true_label][predicted_label] += 1
    return confusion_matrix

In [22]:
def initialize_centroids_with_features(images, labels, num_clusters):
    # Preprocesamiento de las imágenes
    flattened_images = images.reshape(images.shape[0], -1)  # Aplanar las imágenes en vectores de características
    unique_labels = np.unique(labels)

    # Cálculo de características promedio por etiqueta
    centroids = []
    for label in unique_labels:
        label_images = flattened_images[labels == label]
        label_mean = np.mean(label_images, axis=0)
        centroids.append(label_mean)

    # Selección de los primeros K centroides iniciales
    centroids = np.array(centroids)[:num_clusters]

    return centroids

In [23]:
class KMeansRANDOM:
    def __init__(self, k, max_iterations=10000):
        self.k = k
        self.max_iterations = max_iterations
        self.centroids = None
        self.labels = None

    def initialize_centroids(self, data):
        # Inicialización de centroides de manera aleatoria
        np.random.seed(0)
        indices = np.random.choice(data.shape[0], self.k, replace=False)
        centroids = data[indices]
        return centroids

    def assign_clusters(self, data):
        # Asignación de puntos a clústeres según la distancia euclidiana
        distances = np.sqrt(((data[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        return labels

    def update_centroids(self, data):
        # Actualización de los centroides como la media de los puntos asignados a cada clúster
        centroids = np.zeros((self.k, data.shape[1]))
        for i in range(self.k):
            cluster_data = data[self.labels == i]
            if len(cluster_data) > 0:
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fit(self, data):
        self.centroids = self.initialize_centroids(data)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break

    def predict(self, data):
        labels = self.assign_clusters(data)
        return labels

# Ejemplo de uso
# Supongamos que tienes un conjunto de datos 'data' de dimensiones (n, d)
# donde 'n' es el número de puntos y 'd' es el número de dimensiones.

# Supongamos que tienes las etiquetas verdaderas de clase en la variable 'true_labels'
true_labels = Y_test

# Especifica el número de clústeres deseado
k = 10

# Crear instancia de la clase KMeans
kmeans = KMeansRANDOM(k)

# Ajustar el modelo a los datos
kmeans.fit(X_train)

# Obtener las etiquetas asignadas a cada punto
predicted_labels = kmeans.predict(X_test)

# Calcular la matriz de confusión
confusion_matrix = calculate_confusion_matrix(true_labels, predicted_labels, num_classes=k)

# Imprimir la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix)

# Calcular la precisión como la suma de las coincidencias diagonales dividida por el número total de puntos
accuracy = np.sum(np.max(confusion_matrix, axis=1)) / len(X_test)

# Imprimir la precisión
print("Precisión de clasificación: {:.2f}%".format(accuracy * 100))


Matriz de Confusión:
[[  0.   1.   0.   0.  83.   0.   0.   0.   0.  94.]
 [ 21.   0.   0.   3.   0.   1.   1.  95.  61.   0.]
 [150.   0.   9.   0.   0.   7.   3.   3.   4.   1.]
 [  0.   0.   8.   0.   0. 161.   9.   5.   0.   0.]
 [  0. 159.   1.   0.   0.   0.   6.   9.   6.   0.]
 [  0.   2.   2.   2.   0.  74.   0. 100.   2.   0.]
 [  0.   1.   1. 175.   0.   0.   0.   2.   1.   1.]
 [  0.   1.   0.   0.   0.   0. 166.   4.   8.   0.]
 [  1.   0. 118.   1.   0.   7.   1.  36.  10.   0.]
 [  0.   0.   3.   0.   0. 144.   5.   4.  24.   0.]]
Precisión de clasificación: 75.79%


In [24]:
class KMeansPreprocess:
    def __init__(self, k, max_iterations=1000):
        self.k = k
        self.max_iterations = max_iterations
        self.centroids = None
        self.labels = None
        
    def initialize_centroids_with_features(self, data, real_labels):
        # Preprocesamiento de las imágenes
        flattened_images = data.reshape(data.shape[0], -1)  # Aplanar las imágenes en vectores de características
        unique_labels = np.unique(real_labels)

        # Cálculo de características promedio por etiqueta
        centroids = []
        for label in unique_labels:
            label_images = flattened_images[real_labels == label]
            label_mean = np.mean(label_images, axis=0)
            centroids.append(label_mean)

        # Selección de los primeros K centroides iniciales
        centroids = np.array(centroids)[:self.k]

        return centroids

    def assign_clusters(self, data):
        # Asignación de puntos a clústeres según la distancia euclidiana
        distances = np.sqrt(((data[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        return labels

    def update_centroids(self, data):
        # Actualización de los centroides como la media de los puntos asignados a cada clúster
        centroids = np.zeros((self.k, data.shape[1]))
        for i in range(self.k):
            cluster_data = data[self.labels == i]
            if len(cluster_data) > 0:
                centroids[i] = np.mean(cluster_data, axis=0)
        return centroids

    def fit(self, data, real_labels):
        self.centroids = self.initialize_centroids_with_features(data, real_labels)

        for _ in range(self.max_iterations):
            prev_centroids = self.centroids.copy()

            self.labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)

            # Comprobar convergencia
            if np.all(prev_centroids == self.centroids):
                break

    def predict(self, data):
        labels = self.assign_clusters(data)
        return labels

In [25]:
true_labels = Y_test

# Especifica el número de clústeres deseado
k = 10

# Crear instancia de la clase KMeans
kmeans = KMeansPreprocess(k)

# Ajustar el modelo a los datos
kmeans.fit(X_train, Y_train)

# Obtener las etiquetas asignadas a cada punto
predicted_labels = kmeans.predict(X_test)

# Calcular la matriz de confusión
confusion_matrix = calculate_confusion_matrix(true_labels, predicted_labels, num_classes=k)

# Imprimir la matriz de confusión
print("Matriz de Confusión:")
print(confusion_matrix)

# Calcular la precisión como la suma de las coincidencias diagonales dividida por el número total de puntos
accuracy = np.sum(np.max(confusion_matrix, axis=1)) / len(X_test)

# Imprimir la precisión
print("Precisión de clasificación: {:.2f}%".format(accuracy * 100))

Matriz de Confusión:
[[176.   0.   0.   0.   2.   0.   0.   0.   0.   0.]
 [  0. 134.  21.   1.   0.   1.   3.   0.   7.  15.]
 [  1.   6. 150.   4.   0.   0.   0.   3.   9.   4.]
 [  0.   1.   1. 158.   0.   2.   0.   8.   5.   8.]
 [  0.   9.   0.   0. 159.   0.   0.   7.   6.   0.]
 [  0.   0.   0.   0.   1. 138.   1.   0.   0.  42.]
 [  1.   3.   0.   0.   0.   0. 176.   0.   1.   0.]
 [  0.   0.   0.   0.   1.   4.   0. 168.   6.   0.]
 [  0.  19.   1.   2.   0.   2.   1.   1. 138.  10.]
 [  0.   3.   0.   3.   5.   5.   0.   7.   3. 154.]]
Precisión de clasificación: 86.31%
